# Pixelwise Segmentation

Use the `elf.segmentation` module for instance segmentation from pixels based on pixel features.

(Work in progress)

## Preparation

In [ ]:
%gui qt5 
import numpy as np

# import napari for data visualisation
import napari

# import the segmentation functionality from elf
import elf.segmentation.multicut as mc
import elf.segmentation.features as feats

In [ ]:
from skimage.data import astronaut
from skimage.color import rgb2lab
from skimage.util import img_as_float

# generate data for this example:
image = astronaut()
shape = image.shape[:-1]

# lab color space feature
image_features = rgb2lab(image).transpose((2, 0, 1))
image_features -= image_features.min()
image_features /= image_features.max()

# concatenate normalized position as feature
pos_x, pos_y = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]))
image_features = np.concatenate(
    [
        image_features,
        (pos_x.astype('float32') / shape[0])[None],
        (pos_y.astype('float32') / shape[1])[None]
    ],
    axis=0
)

In [ ]:
# visualize the input data with napari
viewer = napari.Viewer()
viewer.add_image(image)
viewer.add_image(image_features)

## Segmentation problem

Set up the segmentation problem from pixels.

In [ ]:
# compute a grid graph for the image
grid_graph = feats.compute_grid_graph(shape)

# compute the edge features
mode = 'cosine'  # mode determines the type of distance, either 'l1', 'l2' or 'cosine'
edge_distances = feats.compute_grid_graph_features(grid_graph, image_features, mode)

In [ ]:
print(edge_distances.min(), edge_distances.max())

## Multicut

In [ ]:
# edge features to costs for the multicut
beta = .5  # boundary bias for more (> .5) or less (< .5) over-segmentation
costs = mc.transform_probabilities_to_costs(edge_distances, beta=beta)
print(costs.min(), costs.max())

In [ ]:
# compute the multicut solution
segmentation = mc.multicut_gaec(grid_graph, costs)
segmentation = segmentation.reshape(shape)
print(shape, segmentation.shape)

In [ ]:
viewer = napari.Viewer()
viewer.add_image(image)
viewer.add_labels(segmentation)

## Lifted Multicut

TODO need long range grid graph features

## Mutex Watershed

TODO need long range grid graph features